# Depth Prediction from RGB and Infrared Input

This model predicts a depth image given a rgb and an infrared input image of the same resolution.



## Import the necessary moduls

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import keras # for data generator class

D:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Training data situation
Training data (as well as test data) will lie in directories with the following structure:

<pre>
data
|-- train
    |-- Color
        |-- 1.jpg
        |-- 2.jpg
        ...
        |-- n.jpg
    |-- Infrared
        |-- 1.png
        |-- 2.png
        ...
        |-- n.png
    |-- Depth
        |-- 1.png
        |-- 2.png
        ...
        |-- n.png
|-- test
    |-- Color
    |-- Infrared
    |-- Depth
</pre>

## The Data Generator
Because there are many training and test images, it is reasonable to utilize a data loader, which reads training data batch wise. Because the default keras data loader (`ImageDataGenerator`) does not work with two input parameters, we need to write our own. For this, the tutorial from https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly is utilized.

In [2]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, path_to_data='data/', batch_size=32, image_size=(480,640), shuffle=True):
        self.path_to_data = path_to_data
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.training_size = __get_training_data_size(self.path_to_data)
        self.on_epoch_end()
        
    def __get_training_data_size(self, path_to_data):
        'gets the number of training examples'
        path_color = os.path.join(path_to_data,'train','Color')
        if os.path.isdir(path_color):
            size = len([color for color in os.listdir(path_color) if os.path.isfile(os.path.join(path_color, color))])
            return size
        else:
            return 0
        
    def __len__(self):
        'Number of batches per epoche'
        return int(np.floor(self.training_size / self.batch_size))
    
    def on_epoch_end(self):
        'Update indices (and their ordering) after each epoch'
        # image names start with 1, np.arange(n,m) returns values from n to (m-1)
        self.indices = np.arange(1, len(self.training_size)+1)
        if self.shuffle == True:
            np.random.shuffle(self.indices)
            
    def __data_generation(self, list_images):
        'Generates data of size batch_size' # X = (batch_size, 480, 640, 1)
        X1 = np.empty((self.batch_size, *self.image_size, 3), dtype=np.uint8) # color images
        X2 = np.empty((self.batch_size, *self.image_size), dtype=np.uint16) # ir image
        y = np.empty((self.batch_size, *self.image_size), dtype=np.uint16)  # depth image
        
        # Generate data
        for idx, name in enumerate(list_images):
            # load images in arrays
            img = cv2.imread(os.path.join(self.path_to_data, 'train', 'Color', str(name)+".jpg"), cv2.IMREAD_COLOR)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            X1[idx,] = img.astype(np.uint8)
            
            img = cv2.imread(os.path.join(self.path_to_data, 'train', 'Infrared', str(name)+".png"), cv2.IMREAD_ANYDEPTH)
            X2[idx,] = img.astype(np.uint16)
            
            img = cv2.imread(os.path.join(self.path_to_data, 'train', 'Depth', str(name)+".png"), cv2.IMREAD_ANYDEPTH)
            y[idx,] = img.astype(np.uint16)
        
        return X1, X2, y
    
    def __getitem__(self, index):
        'Generate one batch of data, X1 contains 8-bit RGB images, X2 16-bit infrared images and y corresponding 16-bit depth images'
        # Generate indices of data
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        
        # Generate data
        X1, X2, y = self.__data_generation(indices)
        
        return [X1, X2], y
    
    
            